# Calculate ICE Demand (EVI-Pro)

## Imports

In [ ]:
# Standard geospatial inputs
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from fiona.crs import from_epsg

# Ignore warnings
import warnings
warnings.simplefilter("ignore")

# Custom utilities
from src.components import Vehicle
from src.simulation import Simulation
from src.grid import HexGrid
from src.models import Random_Sample_Charge_Location_Model, Linear_Kwh_Model

from geojson.feature import *
from h3 import h3
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear
import folium
import json
from pandas.io.json import json_normalize


import seaborn as sns
import matplotlib.pyplot as plt
from src.h3_utils import *

## Load Trajectories
* Alter this to import John's moving pandas pickle files

In [ ]:
data = pd.read_csv('../data/raw/part-00167-tid-8316490494947456795-71092cdb-87a8-47f5-9d32-296121158b22-3052-1.c000.csv')
data = data[data.msa_name == 'Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)']

In [ ]:
data = data[data.battery_pack.isna()]
dataGeo = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.decr_lng, data.decr_lat), crs=from_epsg(4326))
dataGeo['t'] = pd.to_datetime(dataGeo.element_time_local, utc=True)
dataGeo = dataGeo.set_index('t')
traj_collection = mpd.TrajectoryCollection(dataGeo, 'hashed_vin')

## Create Vehicles

In [ ]:
# For each vehicle trajectory create a vehicle object
vehicles = []
for traj in traj_collection.trajectories:
    vehicles.append(Vehicle(traj))

## Create and Train Models

In [ ]:
# Train the linear model
linear_kwh_model = Linear_Kwh_Model()
linear_kwh_model.train()

In [ ]:
# No training required for this model
random_sampling_model = Random_Sample_Charge_Location_Model()

## Run Simulation

In [ ]:
# Select the number of vehicles to be run through simulation
num_vehicles = 100

# Create the simulation
sim = Simulation(vehicles[0:num_vehicles], random_sampling_model, linear_kwh_model)

In [ ]:
# Run the simulation
sim.run()

## See results and visualize

In [ ]:
sim.map(hour=12)